In [1]:
import gym
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow import keras
from ns3gym import ns3env

/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sheila/

In [2]:
# ---- m_collisionTh = 7

env = gym.make('ns3-v0') # environment ID

# numero de estados
s_num = env.observation_space.shape[0]

# numero de acoes
a_num = env.action_space.n

print("\n")
print("Numero de Estados: ", s_num)
print("Numero de Acoes: ", a_num) 

"""
# ambiente anterior
Q = np.array(np.zeros([4,4]))
"""

# ambiente atual
Q_table = np.zeros([s_num, a_num])

Got new port for ns3gm interface:  5696


Numero de Estados:  4
Numero de Acoes:  4


In [3]:
def choose_action(s, s_num, epsilon):
# Selecione a ação com o maior valor Q.
    if np.random.uniform() < epsilon:
        a = int(np.max(Q_table[np.argwhere(s)[0][0]]))
# Selecione uma ação aleatóriamente.
    else:
        a = np.random.randint(0, s_num)
    return a

In [4]:
numTimeSlots = 98
env._max_episode_steps = numTimeSlots

discount_rate = 0.95  # gamma
# learning rate
learning_rate0 = 0.05 # 0.05 # alpha
learning_rate_decay = 0.1 # 0.1 alpha_decay

# exploration rate
epsilon = 0.5               
epsilon_min = 0.01
epsilon_decay = 0.999

# s - Current State
s = env.reset()
s = np.reshape(s, [1, s_num])


for iteration in range(0,numTimeSlots):
    
    
    # a - Action   
    a = choose_action(s, s_num, epsilon) # acao
    #print(isinstance(a, int))
    
    # Step
    next_state, reward, done, _ = env.step(a) # toma uma ação  no ambiente a cada passo de tempo
    #print("current state: ", s, "next state: ", next_state)
    
    # o Next State retornado do env fica no formato [0, 0, 0, 0]
    # é necesário que o Next State fique no formato de vector sem virgula. 
    next_state = np.array(next_state)
    #next_state = np.reshape(next_state, [1, s_num])
    #print("next state array: ", next_state)
    
    # state index
    s_index   = np.argwhere(s==1)[0][0] 
        
    # next-state index
    n_s_index   = np.argwhere(next_state==1)[0][0]
    #print("state index: ", s_index, "next state index: ", n_s_index)            
        
        
    # next-state Q-valor Maximo
    #n_s_Qmax = np.max(Q_table[np.argwhere(next_state==1)[0][0]]) 
    n_s_Qmax = np.max(Q_table[n_s_index])
    
       
    print("Time Slot    :", iteration)
    print("----------------------", "\n")
    print("Current State:", s, "-->", s_index)
    print("Action       :", a)
    print("Next state   :", next_state, "-->", n_s_index) #, "-->", (s + 1) % 4
    print("Reward       :", reward)
    print("Gameover     :", done)
    print("----------------------", "\n")   
   
    
    """
    print('-------- S ---------------')
    print(s)
    print('S_index ', s_index)
    
    print('-------- next_state ---------------')
    print(next_state)
    print(Q_table[np.argwhere(next_state==1)])
    print('ns_index ', n_s_index)
    """
    
    learning_rate = learning_rate0 / (1 + iteration * learning_rate_decay)# passo de aprendizagem
    
    # Calculates and updates the Q-table values
    Q_table[s_index, a] = (1 - learning_rate) * Q_table[s_index, a] + learning_rate * (
        reward + discount_rate * n_s_Qmax)
    
    # updates the current state
    s = next_state
    
    # updates the exploration rate
    if epsilon > epsilon_min: epsilon *= epsilon_decay 
        
        
    
    print("Tabela do Q-value Maximo")
    print(Q_table, "\n")
    print("Acao otima para cada canal:", np.argmax(Q_table, axis=1), "\n")
    print("---------------------------", "\n")
    
   

Time Slot    : 0
---------------------- 

Current State: [[1 0 0 0]] --> 0
Action       : 3
Next state   : [0 1 0 0] --> 1
Reward       : 1.0
Gameover     : False
---------------------- 

Tabela do Q-value Maximo
[[0.   0.   0.   0.05]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]] 

Acao otima para cada canal: [3 0 0 0] 

--------------------------- 

Time Slot    : 1
---------------------- 

Current State: [0 1 0 0] --> 1
Action       : 0
Next state   : [0 0 1 0] --> 2
Reward       : 1.0
Gameover     : False
---------------------- 

Tabela do Q-value Maximo
[[0.         0.         0.         0.05      ]
 [0.04545455 0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]] 

Acao otima para cada canal: [3 0 0 0] 

--------------------------- 

Time Slot    : 2
---------------------- 

Current State: [0 0 1 0] --> 2
Action       : 2
Next state   : [0 0 0 1] --> 3
Reward       : 1.0
Gameover  

Time Slot    : 73
---------------------- 

Current State: [0 1 0 0] --> 1
Action       : 0
Next state   : [0 0 1 0] --> 2
Reward       : 1.0
Gameover     : False
---------------------- 

Tabela do Q-value Maximo
[[ 0.14641435 -0.010727    0.01067468  0.10830889]
 [ 0.20909539  0.0151149  -0.01771254  0.00738948]
 [ 0.12600032  0.04897837  0.07526606 -0.00704074]
 [-0.12400776  0.03235244  0.04469368  0.03971744]] 

Acao otima para cada canal: [0 0 0 2] 

--------------------------- 

Time Slot    : 74
---------------------- 

Current State: [0 0 1 0] --> 2
Action       : 0
Next state   : [0 0 0 1] --> 3
Reward       : 1.0
Gameover     : False
---------------------- 

Tabela do Q-value Maximo
[[ 0.14641435 -0.010727    0.01067468  0.10830889]
 [ 0.20909539  0.0151149  -0.01771254  0.00738948]
 [ 0.13145543  0.04897837  0.07526606 -0.00704074]
 [-0.12400776  0.03235244  0.04469368  0.03971744]] 

Acao otima para cada canal: [0 0 0 2] 

--------------------------- 

Time Slot    : 75
----